This notebook combines the four beams together to create an RBE weighted dose map using the Wedenberg model.

In [ ]:
#Install the Pydiocom Library
!pip install pydicom

     |████████████████████████████████| 1.9MB 7.6MB/s 


In [ ]:
#Import the pydicom library to access the data from dicom files
import pydicom
import numpy as np

In [ ]:
def ProtonRBEWberg(dose,LET,ab):
  '''The function determines the proton RBE using the Wedenberg empirical model.
  The inputs are proton dose per fraction, the voxel's proton LET and the
  alpha/beta ratio of the tissue. Returns the RBE of the proton.
  '''
  #Use linear LET model to find the max RBE
  max = 1 + 0.434*(LET/ab)
  #Assumes min RBE is one
  min = 1

  if dose == 0:
    RBE = min
  else:
    #Use linear quadratic model solution to find RBE
    RBE = (1/dose)*np.sqrt((1/4)*(ab**2) + max*ab*dose + min*(dose**2)) -ab/(2*dose)
    
  return RBE

In [ ]:
#Open the LET files
LETBase = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/LET_Beam40_S1.dcm")
LET40  = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/LET_Beam40_S1.dcm")
LET160 = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/LET_Beam160_S1.dcm")
LET200 = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/LET_Beam200_S1.dcm")
LET320 = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/LET_Beam320_S1.dcm")

#Open the Proton Dose Files
PDoseBase = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam40_S2.dcm")
PDose40  = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam40_S2.dcm")
PDose160 = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam160_S2.dcm")
PDose200 = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam200_S2.dcm")
PDose320 = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam320_S2.dcm")

In [ ]:
#This script determines the RBE-weighted dose distributions for all 4 beams 
#using the Wedenbergy model. It then combines the 4 beams into one image.

scale = 10**-9
array = PDoseBase.pixel_array
ab=2.5

#Iterate over each voxel
for z in range(LETBase.NumberOfFrames):
  for x in range(LETBase.Rows):
    for y in range(LETBase.Columns):

      #First for the 40 degree beam
      dose40 =1000*PDose40.pixel_array[z][x][y]*PDose40.DoseGridScaling
      le40 = LET40.pixel_array[z][x][y]*LET40.DoseGridScaling
      RBE40 = dose40*ProtonRBEWberg(dose40,le40,ab)

      #Then the 160 degree beam
      dose160 =1000*PDose160.pixel_array[z][x][y]*PDose160.DoseGridScaling
      le160 = LET160.pixel_array[z][x][y]*LET160.DoseGridScaling
      RBE160 = dose160*ProtonRBEWberg(dose160,le160,ab)

      #Then the 200 degree beam
      dose200 =1000*PDose200.pixel_array[z][x][y]*PDose200.DoseGridScaling
      le200 = LET200.pixel_array[z][x][y]*LET200.DoseGridScaling
      RBE200 = dose200*ProtonRBEWberg(dose200,le200,ab)

      #Finally the 320 degree beam
      dose320 =1000*PDose320.pixel_array[z][x][y]*PDose320.DoseGridScaling
      le320 = LET320.pixel_array[z][x][y]*LET320.DoseGridScaling
      RBE320 = dose320*ProtonRBEWberg(dose320,le320,ab)

      #Then the four beams are combined
      array[z][x][y]= (1/scale)*(RBE40 + RBE160 + RBE200 + RBE320)

#Save the new DICOM image
PDoseBase.PixelData = array.tobytes()     
PDoseBase.save_as("/content/drive/MyDrive/RBEscale/LETScale_Brainstem_Seed2.dcm")